In [1]:
import pyodim
import hdr, hsda, common
import mesh_ppi_new as mesh

imports static in hsda
imports static in mesh


In [2]:
filename = "/home/jsoderho/Downloads/66_20231110_063000.pvol.h5"
radar_datasets = pyodim.read_odim(filename, lazy_load=False)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [3]:
#run retreivals that work on individual sweeps
for sweep in range(len(radar_datasets)):
    #add HCA data
    hca_meta = common.get_odim_ncar_hca(radar_datasets[sweep]['elevation'].data[0], filename, radar_datasets[sweep]['DBZH'].shape, skip_birdbath=True)
    radar_datasets[sweep] = radar_datasets[sweep].merge(
        {'hca': (("azimuth", "range"), hca_meta['data']) })
    print()
    radar_datasets[sweep] = common.add_pyodim_sweep_metadata(radar_datasets[sweep], 'hca', hca_meta)
    #hail retrievals
    hdr_meta, hdr_size_meta = hdr.main(radar_datasets[sweep]['DBZH'].values,
                                       radar_datasets[sweep]['ZDR'].values)
    hsda_meta = hsda.main(radar_datasets[sweep]['DBZH'].values,
                                        radar_datasets[sweep]['ZDR'].values,
                                        radar_datasets[sweep]['RHOHV'].values,
                                        radar_datasets[sweep]['hca'].values,
                                        radar_datasets[sweep]['z'].values,
                                        [4000,5500],
                                        [6, 7, 8])

    #add new fields
    radar_datasets[sweep] = radar_datasets[sweep].merge(
        {'hdr': (("azimuth", "range"), hdr_meta['data']),
         'hdr_size': (("azimuth", "range"), hdr_size_meta['data']),
         'hsda': (("azimuth", "range"), hsda_meta['data']) })
    #update metadata for new fields
    radar_datasets[sweep] = common.add_pyodim_sweep_metadata(radar_datasets[sweep], 'hdr', hdr_meta)
    radar_datasets[sweep] = common.add_pyodim_sweep_metadata(radar_datasets[sweep], 'hdr_size', hdr_meta)
    radar_datasets[sweep] = common.add_pyodim_sweep_metadata(radar_datasets[sweep], 'hsda', hsda_meta)

#run retreivals that work on entire volume only
#mesh
#build coords
reflectivity_dataset = []
elevation_dataset = []
azimuth_dataset = []
range_dataset = []
radar_altitude = radar_datasets[0].attrs['height']
for sweep_idx in range(len(radar_datasets)):
    reflectivity_dataset.append(radar_datasets[sweep_idx]['DBZH'].values)
    elevation_dataset.append(radar_datasets[sweep_idx]['elevation'].data[0])
    azimuth_dataset.append(radar_datasets[sweep_idx]['azimuth'].values)
    range_dataset.append(radar_datasets[sweep_idx]['range'].values)

print(elevation_dataset)

ke_dict, shi_dict, mesh_dict, posh_dict = mesh.main(reflectivity_dataset,
                                                    elevation_dataset,
                                                    azimuth_dataset,
                                                    range_dataset,
                                                    radar_altitude,
                                                    [4000,5500])















[0.5, 0.8, 1.4, 2.4, 3.5, 4.7, 6.0, 7.8, 10.0, 13.0, 17.0, 23.0, 32.0, 90.0]


ValueError: operands could not be broadcast together with shapes (1490,1490) (360,1490) 

In [9]:
import numpy as np
print(len(2))

TypeError: object of type 'int' has no len()

: 